Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/media/sf_projects/lambda/DS-Unit-2-Applied-Modeling/module2/instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

aisles.csv
departments.csv
orders.csv
order_products__prior.csv
order_products__train.csv
products.csv


### For each csv file, look at its shape & head 

In [7]:
import numpy as np
import pandas as pd
from IPython.display import display

for filename in glob("*.csv"):
    df = pd.read_csv(filename)
    print(filename, df.shape)
    display(df.head())
    

aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


orders.csv (3421083, 7)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [20]:
prior = pd.read_csv("order_products__prior.csv")
most_freq_ordered = prior["product_id"].mode()[0]

In [21]:
products_df = pd.read_csv("products.csv")
products_df[products_df["product_id"] == most_freq_ordered]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [22]:
prior = pd.merge(prior, products_df, on="product_id")
prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [76]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [77]:
orders["next_order_id"] = orders.groupby("user_id")["order_id"].shift(-1)
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,next_order_id
0,2539329,1,prior,1,2,8,NaN,2398795.0
1,2398795,1,prior,2,3,7,15.0,473747.0
2,473747,1,prior,3,3,12,21.0,2254736.0
3,2254736,1,prior,4,4,7,29.0,431534.0
4,431534,1,prior,5,4,15,28.0,3367565.0


In [78]:
train = pd.read_csv("order_products__train.csv")

In [79]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [80]:
train["has_bananas"] = train["product_id"] == 24852
order_has_bananas = train.groupby("order_id")["has_bananas"].any()

In [81]:
order_has_bananas.sum()


18726

In [82]:
order_has_bananas.value_counts(normalize=True)

False    0.857281
True     0.142719
Name: has_bananas, dtype: float64

In [83]:
order_has_bananas.reset_index().head()

,order_id,has_bananas
0,1,False
1,36,False
2,38,False
3,96,False
4,98,False


## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [84]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,next_order_id
0,2539329,1,prior,1,2,8,NaN,2398795.0
1,2398795,1,prior,2,3,7,15.0,473747.0
2,473747,1,prior,3,3,12,21.0,2254736.0
3,2254736,1,prior,4,4,7,29.0,431534.0
4,431534,1,prior,5,4,15,28.0,3367565.0


In [110]:
orders = pd.merge(orders, order_has_bananas.reset_index(), on="order_id")
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,next_order_id,has_bananas_x,has_bananas_y,has_bananas
0,1187899,1,train,11,4,8,14.0,NaN,False,False,False
1,1492625,2,train,15,1,11,30.0,NaN,True,True,True
2,2196797,5,train,5,0,11,6.0,NaN,False,False,False
3,525192,7,train,21,2,11,6.0,NaN,False,False,False
4,880375,8,train,4,1,14,10.0,NaN,False,False,False


In [114]:
orders["has_bananas"].value_counts()

False    112483
True      18726
Name: has_bananas, dtype: int64

In [115]:
banana_orders = orders[orders["has_bananas"] == True]
banana_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,next_order_id,has_bananas_x,has_bananas_y,has_bananas
1,1492625,2,train,15,1,11,30.0,NaN,True,True,True
17,698604,34,train,6,4,13,30.0,NaN,True,True,True
22,1864787,43,train,12,5,11,26.0,NaN,True,True,True
24,1947848,46,train,20,4,20,11.0,NaN,True,True,True
32,13318,56,train,13,1,9,6.0,NaN,True,True,True


In [116]:
banana_orders = banana_orders.drop(columns=["has_bananas_x", "has_bananas_y"])
banana_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,next_order_id,has_bananas
1,1492625,2,train,15,1,11,30.0,NaN,True
17,698604,34,train,6,4,13,30.0,NaN,True
22,1864787,43,train,12,5,11,26.0,NaN,True
24,1947848,46,train,20,4,20,11.0,NaN,True
32,13318,56,train,13,1,9,6.0,NaN,True


## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [117]:
orders_user_buys_bananas = orders[orders["user_id"].isin(set(banana_orders["user_id"]))]
orders_user_buys_bananas.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,next_order_id,has_bananas_x,has_bananas_y,has_bananas
1,1492625,2,train,15,1,11,30.0,NaN,True,True,True
17,698604,34,train,6,4,13,30.0,NaN,True,True,True
22,1864787,43,train,12,5,11,26.0,NaN,True,True,True
24,1947848,46,train,20,4,20,11.0,NaN,True,True,True
32,13318,56,train,13,1,9,6.0,NaN,True,True,True


In [118]:
orders_user_buys_bananas["has_bananas"].value_counts()

True    18726
Name: has_bananas, dtype: int64

In [102]:
orders_user_buys_bananas["is_next_bananas"] = orders_user_buys_bananas.groupby("user_id")["has_bananas"].shift(1)
orders_user_buys_bananas.tail(25)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,next_order_id,has_bananas,is_next_bananas
3417452,2633656,205981,train,7,0,16,30.0,NaN,True,NaN
3417545,262066,205983,train,82,3,11,2.0,NaN,True,NaN
3417918,1689133,206015,train,13,0,21,6.0,NaN,True,NaN
3417979,805424,206018,train,17,6,18,4.0,NaN,True,NaN
3417993,2301058,206019,train,14,5,16,20.0,NaN,True,NaN
3418069,3243,206024,train,43,3,15,2.0,NaN,True,NaN
3418116,2162996,206028,train,14,0,10,24.0,NaN,True,NaN
3418190,1934283,206033,train,6,2,10,8.0,NaN,True,NaN
3418682,1399803,206058,train,5,1,8,7.0,NaN,True,NaN
3418984,86120,206082,train,9,1,8,7.0,NaN,True,NaN


In [92]:
banana_orders = orders_user_buys_bananas["has_bananas"].sum()

18726

In [95]:
(orders_user_buys_bananas["has_bananas"] & orders_user_buys_bananas["is_next_bananas"]).sum()

0

## What features can we engineer? We want to predict, will these customers reorder this product on their next order?